In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [8]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'kdd-missing-model'
id_name = 'id'
target = 'is_exciting_20'

file_name = "kdd-missing-test.csv"
test_file_name = "KDD-missing-test-no-label.csv"

kaggle_file_name = ""
kaggle_test_file_name = ""
kaggle_pred_file_name = ""

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

  secondary_focus_subject primary_focus_subject  school_latitude    cost  \
0    Literature & Writing              Literacy        34.922909  226.19   
1           Special Needs           Visual Arts        63.327567  398.29   
2                     NaN              Literacy        37.640102  749.95   
3         Extracurricular         Special Needs        40.814871  301.93   
4                     NaN         Special Needs        44.094439  476.11   

   school_longitude  Vol_teacher_acctid resource_type  \
0        -86.712257                   1    Technology   
1       -142.987579                   3      Supplies   
2       -118.960945                   6    Technology   
3        -73.905355                   2         Other   
4        -70.206641                   7    Technology   

  teacher_teach_for_america  Vol_school_city  school_zip  ...  \
0                         f                7           0  ...   
1                         f                3           0  ...   
2    

In [11]:
f = open(test_file_name)
count = 0
pred_label = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    #print(response)
    pred_label.append(int(response[0]))
    ind1 = response.index("[")
    ind2 = response.index(",", ind1)
    neg_prob = float(response[ind1+1:ind2])
    #print(neg_prob)
    pred_neg_prpb.append(neg_prob)
    ind1 = response.index("]", ind2)
    pos_prob = float(response[ind2+2:ind1])
    #print(pos_prob)
    pred_pos_prpb.append(pos_prob)
    count = count + 1


In [13]:
print(pred_label[:20])
print(pred_neg_prpb[:20])
print(pred_pos_prpb[:20])


import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred_label)
print("test accuracy = {}".format(np.mean(y_true==y_pred)))

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred_pos_prpb, pos_label=1)
print("test auc = {}".format(metrics.auc(fpr, tpr)))

[1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
[0.4833853244781494, 0.49245959520339966, 0.785319983959198, 0.47368091344833374, 0.7433342933654785, 0.44757533073425293, 0.7421001195907593, 0.6170697510242462, 0.8312297761440277, 0.5649320483207703, 0.7586535513401031, 0.626697838306427, 0.7636849880218506, 0.5302917063236237, 0.6658933460712433, 0.633270651102066, 0.32318103313446045, 0.5592995882034302, 0.4304666519165039, 0.9085303246974945]
[0.5166146755218506, 0.5075404047966003, 0.214680016040802, 0.5263190865516663, 0.2566657066345215, 0.5524246692657471, 0.2578998804092407, 0.3829302489757538, 0.1687702238559723, 0.43506795167922974, 0.24134644865989685, 0.373302161693573, 0.23631501197814941, 0.46970829367637634, 0.3341066539287567, 0.36672934889793396, 0.6768189668655396, 0.4407004117965698, 0.5695333480834961, 0.0914696753025055]
test accuracy = 0.7109317495367903
test auc = 0.7144683065669621


In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

   RESOURCE  MGR_ID  ROLE_ROLLUP_1  ROLE_ROLLUP_2  ROLE_DEPTNAME  ROLE_TITLE  \
0     78766   72734         118079         118080         117878      117879   
1     40644    4378         117961         118327         118507      118863   
2     75443    2395         117961         118300         119488      118172   
3     43219   19986         117961         118225         118403      120773   
4     42093   50015         117961         118343         119598      118422   

   ROLE_FAMILY_DESC  ROLE_FAMILY  ROLE_CODE  
0            118177        19721     117880  
1            122008       118398     118865  
2            301534       249618     118175  
3            136187       118960     120774  
4            300136       118424     118425  
(58921, 9)


In [14]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [21]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

   id  ACTION
0   1       1
1   2       1
2   3       1
3   4       1
4   5       1
(58921, 2)


In [22]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:51:25 GMT'},
  'RetryAttempts': 0}}